In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import xgboost as xgb
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import LabelEncoder,OneHotEncoder,MinMaxScaler,StandardScaler

from sklearn.metrics import r2_score

import matplotlib.pyplot as plt
import seaborn as sbn

data=pd.read_csv('data.csv',encoding='gbk')

real_data=pd.read_csv('sub_a_913.csv')

In [2]:

#根据分析，可以将数据属性分为类别特征和连续型特征
#类别特征
category_feauture=[ 'rentType', 'houseType', 'houseFloor','houseToward', 'houseDecoration', 'communityName', 'region',
                    'plate', 'buildYear', 'room_lobby', 'room_toilet', 'lobby_toilet']

#连续型特征
Value_feauture=[f for f in data.columns if f not in category_feauture and f not in ['ID','tradeTime','tradeMoney','type','city']]
print(Value_feauture)

target=['tradeMoney']

for f in category_feauture:
    ln=LabelEncoder()
    data[f]=ln.fit_transform(data[f])
'''   
for f in Value_feauture:
    if f not in ['ave_money']:
        mms=MinMaxScaler(feature_range=(0,1))
        data[f]=mms.fit_transform(data[f].values.reshape(-1,1))''' 

feas=category_feauture+Value_feauture

['area', 'totalFloor', 'saleSecHouseNum', 'totalTradeMoney', 'totalTradeArea', 'tradeMeanPrice', 'tradeSecNum', 'totalNewTradeMoney', 'totalNewTradeArea', 'tradeNewMeanPrice', 'tradeNewNum', 'remainNewNum', 'supplyNewNum', 'supplyLandNum', 'supplyLandArea', 'tradeLandNum', 'tradeLandArea', 'landTotalPrice', 'landMeanPrice', 'totalWorkers', 'newWorkers', 'residentPopulation', 'pv', 'uv', 'lookNum', 'month', 'days', 'room_num', 'lobby_num', 'toilet_num', 'transport', 'school', 'hospital', 'mall', 'other', 'communityName_count', 'totalFloor_count', 'buildYear_count', 'plate_count', 'totalFloorcommunityName_count', 'buildYearcommunityName_count', 'plateregion_count', 'ave_money', 'community_mean_area', 'community_std_area', 'community_mean_ave_money', 'community_std_ave_money', 'plate_mean_ave_money_x', 'plate_std_ave_money_x', 'plate_mean_area', 'plate_std_area', 'plate_mean_ave_money_y', 'plate_std_ave_money_y', 'community_mean_transport', 'community_std_transport', 'community_mean_mall'

In [3]:
'''from sklearn.mixture import GaussianMixture
def cluster(data):
    cols=[ 'rentType', 'houseType', 'houseFloor','houseToward', 'houseDecoration', 'communityName', 'region',
          
         'plate', 'buildYear', 'room_lobby', 'room_toilet', 'lobby_toilet', 
          
        'totalFloor', 'saleSecHouseNum', 'totalTradeMoney', 'totalTradeArea', 'tradeMeanPrice', 'tradeSecNum', 
          
          'totalNewTradeMoney', 'totalNewTradeArea', 'tradeNewMeanPrice', 'tradeNewNum', 'remainNewNum', 'supplyNewNum',
          
          'supplyLandNum', 'supplyLandArea', 'tradeLandNum', 'tradeLandArea', 'landTotalPrice',
          
          'landMeanPrice', 'totalWorkers', 'newWorkers', 'residentPopulation', 'pv', 'uv', 'lookNum',
          
          'month', 'days', 'room_num', 'lobby_num', 'toilet_num', 'transport', 'school', 'hospital', 'mall',
          
          'other', 'communityName_count', 'totalFloor_count','buildYear_count', 'plate_count',
          
          'totalFloorcommunityName_count', 'buildYearcommunityName_count', 'plateregion_count']
    
    gmm=GaussianMixture(n_components=3, covariance_type='full', random_state=0)
    data['cluster']=pd.DataFrame(gmm.fit(data[cols]).predict(data[cols]))
    
    col1=['rentType', 'houseType', 'houseFloor','houseToward', 'houseDecoration', 'communityName', 'region',
          
         'plate', 'buildYear', 'room_lobby', 'room_toilet', 'lobby_toilet']
    
    col2=['totalFloor', 'saleSecHouseNum', 'totalTradeMoney', 'totalTradeArea', 'tradeMeanPrice', 'tradeSecNum', 
          
          'totalNewTradeMoney', 'totalNewTradeArea', 'tradeNewMeanPrice', 'tradeNewNum', 'remainNewNum', 'supplyNewNum',
          
          'supplyLandNum', 'supplyLandArea', 'tradeLandNum', 'tradeLandArea', 'landTotalPrice',
          
          'landMeanPrice', 'totalWorkers', 'newWorkers', 'residentPopulation', 'pv', 'uv', 'lookNum',
          
          'month', 'days', 'room_num', 'lobby_num', 'toilet_num', 'transport', 'school', 'hospital', 'mall',
          
          'other', 'communityName_count', 'totalFloor_count','buildYear_count', 'plate_count',
          
          'totalFloorcommunityName_count', 'buildYearcommunityName_count', 'plateregion_count']
    
    for f1 in col1:
        for f2 in col2:
            temp=data.groupby(['cluster',f1])[f2].agg('mean').reset_index(name=f1+'feature'+f2)
            temp.fillna(0,inplace=True)
            data=data.merge(temp,on=['cluster',f1],how='left')
            
    return data

data=cluster(data)
#连续型特征
Value_feauture=[f for f in data.columns if f not in category_feauture and f not in ['ID','tradeTime','tradeMoney','type','city']]

print(Value_feauture)
'''
#做log平滑，与LabelEncoder 相比，log平滑线下下降，但线上有提高

'''for f in Value_feauture:
    if f not in ['ave_money']:
        mms=MinMaxScaler(feature_range=(0,1))
        data[f]=mms.fit_transform(data[f].values.reshape(-1,1))
'''
for f in Value_feauture:
    data[f]=data[f].map(lambda x:np.log1p(x))
    
feas=category_feauture+Value_feauture


In [4]:
#划分数据集

train_=data[data['type']=='train'][feas]
test_=data[data['type']=='test'][feas]
train_target=data[data['type']=='train']['tradeMoney']
test_target=data[data['type']=='test']['tradeMoney']

#划分数据为训练集和验证集，比例为 8:2

train_x,valid_x,train_y,valid_y=train_test_split(train_,train_target,test_size=0.2,random_state=42)

# LGB

In [5]:
#构建baseline模型
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'mae',
    'num_leaves': 63,
    'learning_rate': 0.005679442571576396,
    'feature_fraction': 0.7676518091514808,
    'bagging_fraction': 0.9168952116883595,
    'bagging_seed':0,
    'bagging_freq': 1,
    'verbose': 1,
    'reg_alpha':1,
    'reg_lambda':2,
    'min_child_weight':6
}

lgb_train=lgb.Dataset(train_x,train_y)
lgb_eval=lgb.Dataset(valid_x,valid_y,reference=lgb_train)

gbm=lgb.train(params,lgb_train,num_boost_round=10000,valid_sets=[lgb_train,lgb_eval],
                valid_names=['train','valid'],
                early_stopping_rounds=200,
                verbose_eval=1000,
                )
valid1=gbm.predict(valid_x)
feature_rank=gbm.feature_importance()
features_df=pd.DataFrame({ 'column':feas,'importance': feature_rank}).sort_values(by='importance', ascending=False)
print(features_df)

Train until valid scores didn't improve in 200 rounds.
[1000]	train's l1: 558.359	valid's l1: 664.094
[2000]	train's l1: 477.118	valid's l1: 622.508
[3000]	train's l1: 431.526	valid's l1: 607.038
[4000]	train's l1: 398.448	valid's l1: 599.168
[5000]	train's l1: 371.725	valid's l1: 594.915
[6000]	train's l1: 349.648	valid's l1: 592.72
[7000]	train's l1: 330.102	valid's l1: 590.987
[8000]	train's l1: 312.982	valid's l1: 590.134
Early stopping, best iteration is:
[8655]	train's l1: 302.701	valid's l1: 589.75
                                column  importance
12                                area       38335
54                           ave_money       25780
38                                days       25606
56                  community_std_area       22368
55                 community_mean_area       21927
58             community_std_ave_money       20604
57            community_mean_ave_money       16188
13                          totalFloor       14860
17                      tradeM

In [6]:
from sklearn.metrics import r2_score
r2=r2_score(valid1,valid_y)
print("验证集结果:{}".format(r2))

# 未做特征工程前，验证集结果 :0.8480048499945494

#做特征工程，没加方差和均值统计特征前，验证集结果:0.8569968502829308（线下有提高）

#加了方差和均值统计特征，验证集结果:0.8669673740210536（有提高）

#做了聚类特征 验证集结果:0.8654237103479906，线下有点下降

# 验证集结果:0.8659004972100439

验证集结果:0.8679131209717259


# XGB

In [7]:
xgb_model = xgb.XGBRegressor(max_depth=5 , learning_rate=0.05, n_estimators=2000, 
                        objective='reg:gamma', tree_method = 'hist',subsample=0.9, 
                        colsample_bytree=0.7, min_child_samples=5,eval_metric = 'rmse' 
                        )
xgb_model.fit(train_x, train_y,eval_set=[(train_x, train_y),(valid_x, valid_y)], 
              early_stopping_rounds=100, verbose=100)
valid2=xgb_model.predict(valid_x)

D:\Anaconda\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[0]	validation_0-rmse:6213.02	validation_1-rmse:6380.91
Multiple eval metrics have been passed: 'validation_1-rmse' will be used for early stopping.

Will train until validation_1-rmse hasn't improved in 100 rounds.
[100]	validation_0-rmse:6154.55	validation_1-rmse:6323.48
[200]	validation_0-rmse:3498.19	validation_1-rmse:3715.28
[300]	validation_0-rmse:1405.03	validation_1-rmse:1661.64
[400]	validation_0-rmse:1238.33	validation_1-rmse:1515.7
[500]	validation_0-rmse:1157.1	validation_1-rmse:1463.94
[600]	validation_0-rmse:1096.36	validation_1-rmse:1434.19
[700]	validation_0-rmse:1050.21	validation_1-rmse:1423.74
[800]	validation_0-rmse:1007.55	validation_1-rmse:1425.45
Stopping. Best iteration:
[716]	validation_0-rmse:1042.92	validation_1-rmse:1420.52



In [8]:
#验证结果
r2=r2_score(valid2,valid_y)
print("验证集结果:{}".format(r2))
# 验证集结果:0.8504732994179766

验证集结果:0.8504732994179766


# 融合

In [9]:
valid=0.5*valid1+valid2*0.5
#融合验证结果
r2=r2_score(valid,valid_y)
print("验证集结果:{}".format(r2))

验证集结果:0.8679074699532929


In [10]:
#训练整个数据模型，并预测
lgb_train_=lgb.Dataset(train_,train_target)
gbm=lgb.train(params,lgb_train_,num_boost_round=10000,valid_sets=[lgb_train_],
              valid_names=['train'],
              early_stopping_rounds=200,
              verbose_eval=1000)

real_data['pred1']=gbm.predict(test_)
r2=r2_score(real_data['pred1'],real_data['pre'])
print("测试集结果:{}".format(r2))
#测试集结果: 0.9462638018840759

Train until valid scores didn't improve in 200 rounds.
[1000]	train's l1: 568.076
[2000]	train's l1: 489.65
[3000]	train's l1: 445.587
[4000]	train's l1: 413.792
[5000]	train's l1: 388.758
[6000]	train's l1: 368.029
[7000]	train's l1: 350.08
[8000]	train's l1: 334.155
[9000]	train's l1: 319.87
[10000]	train's l1: 306.727
测试集结果:0.953149527789435


In [11]:
xgb_model.fit(train_, train_target)
real_data['pred2']=xgb_model.predict(test_)
r2=r2_score(real_data['pred2'],real_data['pre'])
print("测试集结果:{}".format(r2))
# 测试集结果: 0.9578887129585293

D:\Anaconda\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
D:\Anaconda\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


测试集结果:0.9578887129585293


In [18]:
#融合结果
real_data['pred']=0.4*real_data['pred1']+0.6*real_data['pred2']
r2=r2_score(real_data['pred'],real_data['pre'])
print("测试集结果:{}".format(r2))

#融合之后测试集结果:0.9631897556310717，可以看出平均加权融合有一定提升

测试集结果:0.96454862677646


In [13]:
import catboost as cat

ModuleNotFoundError: No module named 'catboost'